In [4]:
import sys
#sys.path.insert(0, "./src/")

import pdb_numpy
from pdb_numpy import format

In [6]:

coor = format.mmcif.fetch_BioAssembly("8BFT")
coor.write('tmp.cif')
aa_seq = coor.get_aa_seq()
print(aa_seq.keys())

MMCIF file tmp.cif already exist, file not saved


dict_keys(['A', 'B', 'A-2', 'B-2'])


In [4]:
coor = pdb_numpy.Coor("tmp_HFQ_280_600.pdb")

coor.write("tmp_HFQ_280_600_2.pdb")

In [3]:
%load_ext cython

In [4]:
%%cython -a

from libc.string cimport strcpy, strlen

def hy36decode_tmp(int width, str s):
    """ Decodes base-10/upper-case base-36/lower-case base-36 hybrid.
    """

    tmp = s.encode('utf-8')

    cdef:
        char f
        char * number_str
    number_str = tmp

    print(strlen(number_str), width)


cdef int convert_low(char c):
    if c >= 48 and c <=57: #'0':48 '9':57
        return c - 48
    elif c >= 97 and c <=122: #'a':97 'z':122
        #print(chr(c), c - 87)
        return c - 87 # -97 'a' + 10
    raise ValueError("invalid number literal.")

cdef int convert_Cap(char c):
    if c >= 48 and c <=57: #'0':48 '9':57
        return c - 48
    elif c >= 65 and c <=90: #'A':65 'Z':90
        #print(chr(c), c - 87)
        return c - 55 # -65 'A' + 10
    raise ValueError("invalid number literal.")
    
cdef int convert_Hexa_Cap(char c):
    if c == 32: #'0':48
        return 0
    if c >= 48 and c <=57: #'0':48 '9':57
        return c - 48
    elif c >= 65 and c <=90: #'A':65 'Z':90
        #print(chr(c), c - 87)
        return c - 55 # -65 'A' + 10
    raise ValueError("invalid number literal.")
    
cdef int decode_pure_Low(char * s):
    """ Decodes the string s using value associations for each character
    """
    cdef:
        int result = 0, n = 36
        char c
    
    for c in s:
        result *= n
        result += convert_low(c)
    return result


cdef int decode_basic_Hexa_Cap(char * s):
    """ Decodes the string s using value associations for each character
    """
    cdef:
        int result = 0, n = 16
        char c
    
    for c in s:
        print('c:', c)
        result *= n
        result += convert_Hexa_Cap(c)
    return result


cpdef int decode_pure_Cap(char * s):
    "decodes the string s using value associations for each character"
    cdef:
        int result = 0, n = 36
        char c
    
    for c in s:
        result *= n
        result += convert_Cap(c)
    return result

cdef bint isDigit(char c):
    if c >= 48 and c <=57: #'0':48 '9':57
        return True
    else:
        return False

cdef bint isCapLetter(char c):
    if c >= 65 and c <=90: #'A':65 'Z':90
        return True
    else:
        return False

cdef bint isLowLetter(char c):
    if c >= 97 and c <=122: #'a':97 'z':122   
        return True
    else:
        return False
        
def hy36decode(int width, str s):
    """ Decodes base-10/upper-case base-36/lower-case base-36 hybrid.
    """

    tmp = s.encode('utf-8')

    cdef:
        char f
        char * number_str
    number_str = tmp

    print(strlen(number_str), width)

    if strlen(number_str) == width:
        f = number_str[0]
        print('f:', f)
        if f == b"-" or f == b" " or isDigit(f):
            try:
                print('try int(s)')
                return int(s)
            except ValueError:
                print('except:')
                return (
                    decode_basic_Hexa_Cap(number_str)
                )
                pass
            if s == " " * width:
                return decode_pure_Cap
        elif isCapLetter(f):
            try:
                return (
                    decode_pure_Cap(number_str)
                    - 10 * 36 ** (width - 1)
                    + 10**width
                )
            except KeyError:
                pass
        elif isLowLetter(f):
            try:
                return (
                    decode_pure_Low(s=number_str)
                    + 16 * 36 ** (width - 1)
                    + 10**width
                )
            except KeyError:
                pass
        else:
            print('no case matched')
    raise ValueError("invalid number literal.")


In [5]:
convert_Cap("   A".encode('utf-8'))

NameError: name 'convert_Cap' is not defined

In [6]:
line = "HETATMA10AA  O   HOH 2   A      30.843  92.837  48.478  1.00  0.00           O  "
print('resid', line[22:26])
hy36decode(4, line[22:26])


resid    A
4 4
f: 32
try int(s)
except:
c: 32
c: 32
c: 32
c: 65


10

In [39]:
from pdb_numpy.format import encode_cython as encode

resid = "   A"
#encode.hy36decode(4, resid)
tmp = resid.encode('utf-8')
tmp[0]
#hy36decode(4, resid)


32

In [ ]:
coor = pdb_numpy.Coor("./src/pdb_numpy/tests/input/examplefold_pdb_7rce_model_0.cif")

In [9]:
coor.get_aa_na_seq()

{'A': 'MASSRRESINPWILTGFADAEGSFGLSILNRNRGTARYHTRLSFTIMLHNKDKSILENIQSTWKVGSILNNGDHYVSLVVYRFEDLKVIIDHFEKYPLITQKLGDYKLFKQAFSVMENKEHLKENGIKELVRIKAKMNWGLNDELKKAFPENISKERPLINKNIPNFKWLAGFTSGDGSFFVRLRKSNVNARVRVQLVFEISQHIRDKNLMNSLITYLGCGHIYEGNKSERSWLQFRVEKFSDINDKIIPVFQENTLIGVKLEDFEDWCKVAKLIEEKKHLTESGLDEIKKIKLNMNKGR',
 'F': 'GGGGGCATGCAGATCCCACAGGCGCG',
 'G': 'CCGCGCCTGTGGGATCTGCATGCCCC'}

In [10]:
coor_CA_CB = coor.select_atoms("(protein and (name CB or (resname GLY and name CA))) or (dna and name P)")
#coor_CA_CB.chain

In [21]:
lig_chains = None
rec_chains = None
cutoff = 8.0
from pdb_numpy.geom import distance_matrix
import numpy as np
L=0.724,
x0=152.611
k=0.052
b=0.018

model_seq = coor.get_aa_na_seq()

if lig_chains is None:
    lig_chains = [
        min(model_seq.items(), key=lambda x: len(x[1].replace("-", "")))[0]
    ]
assert len(lig_chains) >= 1, "At least one ligand chain is allowed"
print(f'Model ligand chain : {" ".join(lig_chains)}')

if rec_chains is None:
    rec_chains = [chain for chain in model_seq if chain not in lig_chains]
assert len(rec_chains) >= 1, "At least one receptor chain is allowed"
print(f'Model receptor chain : {" ".join(rec_chains)}')

pdockq_list = []

Model ligand chain : F
Model receptor chain : A G


In [22]:

for model in coor_CA_CB.models:
    rec_in_contact = model.select_atoms(
        f"chain {' '.join(rec_chains)} and within {cutoff} of chain {' '.join(lig_chains)}"
    )

    lig_in_contact = model.select_atoms(
        f"chain {' '.join(lig_chains)} and within {cutoff} of chain {' '.join(rec_chains)}"
    )

    dist_mat = distance_matrix(rec_in_contact.xyz, lig_in_contact.xyz)
    contact_num = np.sum(dist_mat < cutoff)

    if contact_num == 0:
        pdockq = 0.0
        pdockq_list.append(pdockq)
        continue

    avg_plddt = rec_in_contact.len * np.average(
        rec_in_contact.beta
    ) + lig_in_contact.len * np.average(lig_in_contact.beta)
    avg_plddt /= rec_in_contact.len + lig_in_contact.len

    x = avg_plddt * np.log10(contact_num)
    pdockq = L / (1 + np.exp(-k * (x - x0))) + b

    pdockq_list.append(pdockq)

print(pdockq_list)


[array([0.5549347])]


In [7]:
from pdb_numpy import DSSP

In [2]:
import sys
import numpy as np
from copy import deepcopy

sys.path.insert(0, "./src/")
from pdb_numpy import Coor
from pdb_numpy import Model
from pdb_numpy import DSSP
from pdb_numpy.format import mmcif, pdb
import pdb_numpy.visu

ImportError: cannot import name 'select' from partially initialized module 'pdb_numpy' (most likely due to a circular import) (/home/murail/Documents/Code/pdb_numpy/./src/pdb_numpy/__init__.py)

In [9]:
test = Coor(pdb_id="3cdc")
view = pdb_numpy.visu.get_py3dmolview(test)
view

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [7]:
test.resid

array([ -1,  -1,  -1, ..., 300, 301, 302], dtype=int32)

In [8]:
test.uniq_resid

array([  0,   0,   0, ..., 617, 618, 619], dtype=int32)

In [13]:
SS_list = DSSP.compute_DSSP(test)

for chain in SS_list[0].keys():
    print(chain, len(SS_list[0][chain]))

A 109
B 108


In [2]:

test = Coor('src/pdb_numpy/tests/input/2ol9.cif')
print(test.len)
test.write("test_raw.pdb", check_file_out=False)

56


In [3]:
test.transformation

{1: {'chains': ['A', 'B'],
  'matrix': [[1.0, 1.0, 0.0, 0.0, 0.0],
   [1.0, 0.0, 1.0, 0.0, 0.0],
   [1.0, 0.0, 0.0, 1.0, 0.0],
   [2.0, 1.0, 0.0, 0.0, 0.9401502764],
   [2.0, 0.0, 1.0, 0.0, 4.7875628934],
   [2.0, 0.0, 0.0, 1.0, 0.0]]}}

In [4]:
test.apply_transformation()
test.write("test_trans.pdb", check_file_out=False)
print(test.len)
#assert test.len == 112

112


In [5]:

# Here there is only identity matrix
# as symmetry, so no change is expected
test.add_symmetry()
print(test.len)
test.write("test_sym.pdb", check_file_out=False)

No symmetry matrix found.


112


In [6]:
test.copy_box(3, 3, 3)
assert test.len == 3024
test.compute_chains_CA()
test.remove_overlap_chain()
assert test.len == 3024

test.write("test.pdb", check_file_out=False)

In [7]:
test.write('tmp.pdb', check_file_out=False)

In [8]:
test_2 = Coor('tmp.pdb')
test_2.write('tmp2.gro', check_file_out=False)

In [10]:
test.add_symmetry()
test.apply_transformation(index=1)
test.compute_chains_CA()
test.write('tmp.pdb', check_file_out=False)

No symmetry matrix found.


In [11]:
test.symmetry

''

In [12]:
test.models

In [13]:
print(test.transformation)

{1: {'chains': ['A', 'B'], 'matrix': [[1.0, 1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 1.0, 0.0, 0.0], [1.0, 0.0, 0.0, 1.0, 0.0], [2.0, 1.0, 0.0, 0.0, 0.9401502764], [2.0, 0.0, 1.0, 0.0, 4.7875628934], [2.0, 0.0, 0.0, 1.0, 0.0]]}}


In [14]:
def parse_transformation(text):

    transformation_dict = {}

    for line in text.split('\n'):
        if line[11:23] == "BIOMOLECULE:":
            biomol = int(line[24:])
            transformation_dict[biomol] = {'chains': [], 'matrix': []}
        elif line[34:41] == "CHAINS:":
            transformation_dict[biomol]['chains'] += line[42:].split()
        elif line.startswith('REMARK 350   BIOMT'):
            transformation_dict[biomol]['matrix'] += [[float(x) for x in line[19:].split()]]
        #print(line, line[34:])
    print(transformation_dict)


parse_transformation(test.transformation)

AttributeError: 'dict' object has no attribute 'split'

In [34]:
test.models.append(deepcopy(test.models[0]))
test.models.append(deepcopy(test.models[0]))

In [4]:
text = \
"""REMARK 350   BIOMT1   2 -1.000000  0.000000  0.000000        0.00000            
REMARK 350   BIOMT2   2  0.000000  1.000000  0.000000        7.05000            
REMARK 350   BIOMT3   2  0.000000  0.000000 -1.000000        0.00000"""

rot_mat = []

for line in text.split("\n"):
    rot_mat.append([float(x) for x in line.split()[4:]])

rot_mat = np.array(rot_mat, dtype=object)
print(rot_mat)

text = \
"""REMARK 350   BIOMT1   3 -1.000000  0.000000  0.000000        0.00000            
REMARK 350   BIOMT2   3  0.000000  1.000000  0.000000       -2.35000            
REMARK 350   BIOMT3   3  0.000000  0.000000 -1.000000        0.00000"""

rot_mat_2 = []

for line in text.split("\n"):
    rot_mat_2.append([float(x) for x in line.split()[4:]])

rot_mat_2 = np.array(rot_mat_2, dtype=object)
print(rot_mat_2)

[[-1.0 0.0 0.0 0.0]
 [0.0 1.0 0.0 7.05]
 [0.0 0.0 -1.0 0.0]]
[[-1.0 0.0 0.0 0.0]
 [0.0 1.0 0.0 -2.35]
 [0.0 0.0 -1.0 0.0]]


In [21]:
rot_mat[:,:3]

array([[-1.0, 0.0, 0.0],
       [0.0, 1.0, 0.0],
       [0.0, 0.0, -1.0]], dtype=object)

In [36]:
test.models[1].xyz = np.dot(test.models[0].xyz, rot_mat[:,:3]) + rot_mat[:,3]
test.models[2].xyz = np.dot(test.models[0].xyz, rot_mat_2[:,:3]) + rot_mat_2[:,3]

In [37]:
test.merge_model()
test.write_pdb('tmp.pdb', check_file_out=False)

In [25]:
pwd

'/home/murail/Documents/Code/pdb_numpy'